In [1]:
import os
import re
import numpy as np
import json
from random import shuffle
from PIL import Image
import matplotlib.pyplot as plt


label = json.load(open('D://CISC351//Group_Project_Data//CCRCC.json','r'))

In [2]:
f = open("cohort.csv","r")
all_dict = {} # id: subcategory
allFiles = f.read()
allFiles = allFiles.split('\n')
allFiles.pop(0)

'"_id","Case_ID","Specimen_ID","Slide_ID","Tumor","Topographic_Site","Specimen_Type","Weight","Tumor_Site","Pathology","Tumor_Histological_Type","Tumor_Segment_Acceptable","Percent_Tumor_Nuclei","Percent_Total_Cellularity","Percent_Necrosis","Normal Segment Free of Tumor","HasRadiology","Radiology","Genomics","Proteomics","Genomics_Available","GDC Link","Proteomics_Available","PDC Link","Gender","Age","Vital status at 12months follow up","Vital status at 24months follow up","Ethnicity","Race","Fully Qualified"'

In [3]:
for i in range(len(allFiles)):
    allFiles[i] = allFiles[i].split(',')
    all_dict[allFiles[i][3]] = allFiles[i][6]

In [4]:
imgDir = os.listdir(".//CCRCC_JPEG")

r = re.compile(".*dzi")

newlist = list(filter(r.match,imgDir))
for d in newlist:
    imgDir.remove(d)

print(len(imgDir))

782


In [5]:
def crop(input):
    im = Image.open(input)
    imgwidth, imgheight = im.size
    
    box = (224, 224, 288, 288)
    a = im.crop(box)
    #o.save(os.path.join(path,"PNG","%s" % page,"IMG-%s.png" % k))
    return a       

def load_training_data():
    train_data = []
    
    k = 1
    #for j in range(len(imgDir)):
    for j in range(0, 102):
        print(k)
        k = k+1
        if (all_dict['"'+imgDir[j].replace('_files','')+'"']=="normal_tissue"):
            label = np.array([1,0])
        else:
            label = np.array([0,1])
        
        for img in os.listdir(".//CCRCC_JPEG//"+imgDir[j]+"//20.0"):
            path = os.path.join(".\\CCRCC_JPEG\\"+imgDir[j]+"\\20.0", img)
            
            #img = Image.open(path)
            img = crop(path)
            train_data.append([np.array(img), label])
            
    
    shuffle(train_data)
    return train_data

In [6]:
def load_test_data():
    train_data = []
    
    k = 1
    #for j in range(len(imgDir)):
    for j in range(0,103):
        print(j)
        # Checking label. Didn't like the string written as "normal_tissue" for whatever reason
        if (all_dict['"'+imgDir[j].replace('_files','')+'"']==all_dict['"'+imgDir[1].replace('_files','')+'"']):
            label = np.array([1,0])
        else:
            label = np.array([0,1])
        
        for img in os.listdir(".//CCRCC_JPEG//"+imgDir[j]+"//20.0"):
            path = os.path.join(".\\CCRCC_JPEG\\"+imgDir[j]+"\\20.0", img)
            img = Image.open(path)
            train_data.append([np.array(img), label])
        
        print(label)
    
    shuffle(train_data)
    return train_data

In [7]:
print(all_dict['"'+imgDir[103].replace('_files','')+'"'])

"normal_tissue"


In [ ]:
# function creates tensors that are too large for memory. Need to figure out another way to build the tensors. Number of images per directory found in the next cell.
im = load_training_data()
plt.imshow(im[0][0])

In [9]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization

model = Sequential()

Using TensorFlow backend.


In [10]:
img_height = 64
img_width = 64

model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(img_height,img_width, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation = 'softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [11]:
trainImages = np.array([i[0] for i in im]).reshape(-1, img_height, img_width, 3)
trainLabels = np.array([i[1] for i in im],dtype = "object") 

In [12]:
trainImages = np.asarray(trainImages).astype(np.int)
trainLabels = np.asarray(trainLabels).astype(np.int)

In [17]:
print(sum(trainLabels))

[    0 35316]


In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [14]:
model.fit(trainImages, trainLabels, batch_size = 50, epochs = 3, verbose = 1)


Epoch 1/3
35316/35316 [==============================] - 32s 913us/step - loss: 0.0013 - accuracy: 0.9991s - loss: 0.0014 -  - ETA: 0s - loss: 0.0013 - accu
Epoch 2/3
35316/35316 [==============================] - 30s 859us/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/3
35316/35316 [==============================] - 30s 854us/step - loss: 0.0000e+00 - accuracy: 1.0000


In [15]:
#im = load_test_data()

In [16]:
print(imkfg[101][1])

NameError: name 'imkfg' is not defined

In [ ]:
trainImages = np.array([i[0] for i in im]).reshape(-1, img_height, img_width, 3)
trainLabels = np.array([i[1] for i in im],dtype = "object") 
trainImages = np.asarray(trainImages).astype(np.int)
trainLabels = np.asarray(trainLabels).astype(np.int)
print(trainImages.shape)

In [ ]:
print(trainLabels)

In [ ]:
loss, acc = model.evaluate(trainImages, trainLabels, verbose = 0)
print(acc * 100)